In [1]:
import gc
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd 
import pickle
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from xgboost import XGBClassifier

In [2]:
data_path = './kaggle/input/geolifeclef-2024'
output_path = './kaggle/working'

In [3]:
# Load the PA metadata for the training set
pa_metadata_train_csv_filename = data_path + '/GLC24_PA_metadata_train.csv'
pa_metadata_train_df = pd.read_csv(pa_metadata_train_csv_filename)

In [4]:
# Get information about each survey for plotting locations, including the number
# of each species found in each survey
pa_survey_metadata_train_df = (pa_metadata_train_df
                               .copy()
                               .drop_duplicates('surveyId'))
pa_survey_metadata_train_df.drop('speciesId', axis=1, inplace=True)
pa_survey_metadata_train_df['speciesCount'] = (pa_metadata_train_df
                                        .groupby('surveyId')
                                        .count()
                                        .speciesId
                                        .tolist())

In [5]:
# Set up the environmental raster file paths
climate_raster_path = data_path + '/EnvironmentalRasters/EnvironmentalRasters/Climate'
elevation_raster_path = data_path + '/EnvironmentalRasters/EnvironmentalRasters/Elevation'
human_raster_path = data_path + '/EnvironmentalRasters/EnvironmentalRasters/Human Footprint'
land_cover_raster_path = data_path + '/EnvironmentalRasters/EnvironmentalRasters/LandCover'
soil_grid_raster_path = data_path + '/EnvironmentalRasters/EnvironmentalRasters/SoilGrids'

# Load the climate raster csv file
average_climate_raster_train_csv_filename = (climate_raster_path + 
                                             '/Average 1981-2010/GLC24-PA-train-bioclimatic.csv')
monthly_climate_raster_train_csv_filename = (climate_raster_path + 
                                             '/Monthly/GLC24-PA-train-bioclimatic_monthly.csv')

average_climate_raster_train_df = pd.read_csv(average_climate_raster_train_csv_filename)
monthly_climate_raster_train_df = pd.read_csv(monthly_climate_raster_train_csv_filename)

pa_elevation_train_csv_filename = (elevation_raster_path + 
                                   '/GLC24-PA-train-elevation.csv')
pa_elevation_train_df = pd.read_csv(pa_elevation_train_csv_filename)

pa_elevation_train_df['has_missing'] = pa_elevation_train_df.isnull().any(axis=1)

pa_human_train_csv_filename = (human_raster_path + 
                               '/GLC24-PA-train-human_footprint.csv')
pa_human_train_df = pd.read_csv(pa_human_train_csv_filename)

pa_human_train_df['has_missing'] = pa_human_train_df.isnull().any(axis=1)

pa_land_cover_train_csv_filename = (land_cover_raster_path + 
                                   '/GLC24-PA-train-landcover.csv')
pa_land_cover_train_df = pd.read_csv(pa_land_cover_train_csv_filename)

pa_soil_grid_train_csv_filename = (soil_grid_raster_path + 
                                   '/GLC24-PA-train-soilgrids.csv')
pa_soil_grid_train_df = pd.read_csv(pa_soil_grid_train_csv_filename)

pa_soil_grid_train_df['has_missing'] = pa_soil_grid_train_df.isnull().any(axis=1)

## Landsat Time Series

In [6]:
landsat_train_path = data_path + '/PA-train-landsat_time_series'

blue_landsat_train_csv_filename = landsat_train_path + '/GLC24-PA-train-landsat_time_series-blue.csv'
green_landsat_train_csv_filename = landsat_train_path + '/GLC24-PA-train-landsat_time_series-green.csv'
red_landsat_train_csv_filename = landsat_train_path + '/GLC24-PA-train-landsat_time_series-red.csv'
nir_landsat_train_csv_filename = landsat_train_path + '/GLC24-PA-train-landsat_time_series-nir.csv'
swir1_landsat_train_csv_filename = landsat_train_path + '/GLC24-PA-train-landsat_time_series-swir1.csv'
swir2_landsat_train_csv_filename = landsat_train_path + '/GLC24-PA-train-landsat_time_series-swir2.csv'

blue_landsat_train_df = pd.read_csv(blue_landsat_train_csv_filename)
green_landsat_train_df = pd.read_csv(green_landsat_train_csv_filename)
red_landsat_train_df = pd.read_csv(red_landsat_train_csv_filename)
nir_landsat_train_df = pd.read_csv(nir_landsat_train_csv_filename)
swir1_landsat_train_df = pd.read_csv(swir1_landsat_train_csv_filename)
swir2_landsat_train_df = pd.read_csv(swir2_landsat_train_csv_filename)

In [7]:
# Initialize the species/survey matrix
max_species_id = pa_metadata_train_df.speciesId.max()
pa_species_df = pd.DataFrame(0, 
                             index=pa_metadata_train_df.surveyId.unique(),
                             columns=list(range(1, int(max_species_id+1))))

In [8]:
# Drop duplicate (speciesId, surveyId) entries
pa_metadata = pa_metadata_train_df.drop_duplicates(['speciesId', 'surveyId']).copy()

# Change the speciesId dtype to int
pa_metadata['speciesId'] = pa_metadata['speciesId'].apply(int)

surveyId_map = {idx: i for i, idx in enumerate(pa_metadata.surveyId.unique())}

# Set the values in the species/survey matrix
pa_species_df.values[pa_metadata.surveyId.map(surveyId_map), 
                     pa_metadata.speciesId.apply(lambda x: x-1).tolist()] = 1

pa_species_df.reset_index(names='surveyId', inplace=True)

In [9]:
# Perform one-hot encoding for 'country' and 'region' variables
pa_survey_metadata_train_df = pd.merge(pa_survey_metadata_train_df,
                                       pd.get_dummies(pa_survey_metadata_train_df[['country', 'region']], dtype='int'),
                                       how='left',
                                       left_index=True,
                                       right_index=True)

In [10]:
# Combine the dataframes into a single, larger dataframe.

# To avoid any potential issues with data leakage I have cut off the monthly climate raster dataframe
# at the end of 2016.

monthly_climate_end_idx = 817

pa_train_df = pd.merge(pa_survey_metadata_train_df.drop(['areaInM2', 'geoUncertaintyInM'], axis=1),
                      monthly_climate_raster_train_df.iloc[:, 0:monthly_climate_end_idx],
                      how='left',
                      on='surveyId')

pa_train_df = pd.merge(pa_train_df,
                      average_climate_raster_train_df,
                      how='left',
                      on='surveyId')

pa_train_df = pd.merge(pa_train_df,
                       pa_elevation_train_df.drop('has_missing', axis=1),
                       how='left',
                       on='surveyId')

pa_train_df = pd.merge(pa_train_df,
                       pa_human_train_df.drop('has_missing', axis=1),
                       how='left',
                       on='surveyId')

pa_train_df = pd.merge(pa_train_df,
                       pa_land_cover_train_df,
                       how='left',
                       on='surveyId')

pa_train_df = pd.merge(pa_train_df,
                       pa_soil_grid_train_df.drop('has_missing', axis=1),
                       how='left',
                       on='surveyId')

In [11]:
def reindex_landsat_data(row, band, year_df):
    year = year_df.loc[year_df.surveyId == row.surveyId, "year"].values[0]
    start_idx = (year - 2017)*4 + 1
    end_idx = start_idx + 68
    obs = row[start_idx:end_idx]
    obs.index = [f"{band}_{lag}" for lag in list(range(68, 0, -1))]
    return obs

In [12]:
landsat_df_list = [blue_landsat_train_df,
                   green_landsat_train_df,
                   red_landsat_train_df,
                   nir_landsat_train_df,
                   swir1_landsat_train_df,
                   swir2_landsat_train_df]

landsat_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']

for band, landsat_df in zip(landsat_bands, landsat_df_list):
    landsat_merge_df = landsat_df.apply(reindex_landsat_data,
                                        axis=1,
                                        band=band,
                                        year_df=pa_survey_metadata_train_df)
    pa_train_df = pd.merge(pa_train_df,
                           landsat_merge_df,
                           how='left',
                           left_index=True,
                           right_index=True)

In [13]:
# Perform spatial interpolation to fill missing values. I will do this by creating a grid and using
# the mean values from the grid boxes to fill any null values
lat_n_gridlines = 201
lon_n_gridlines = 201
lat_min = pa_survey_metadata_train_df.lat.min()
lat_max = pa_survey_metadata_train_df.lat.max()
lon_min = pa_survey_metadata_train_df.lon.min()
lon_max = pa_survey_metadata_train_df.lon.max()

# lat_gridlines = np.linspace(lat_min, lat_max, lat_n_gridlines)
# lon_gridlines = np.linspace(lon_min, lon_max, lon_n_gridlines)
lat_grid = pd.interval_range(lat_min, lat_max, periods=(lat_n_gridlines-1), closed='both')
lon_grid = pd.interval_range(lon_min, lon_max, periods=(lon_n_gridlines-1), closed='both')

In [14]:
def loc_to_grid_num(row):
    lat_num = np.array(range(lat_n_gridlines-1))[lat_grid.contains(row.lat)][0]
    lon_num = np.array(range(lon_n_gridlines-1))[lon_grid.contains(row.lon)][0]
        
    grid_num = lat_num*(lon_n_gridlines - 1) + lon_num
    return grid_num

In [15]:
pa_train_df['grid_num'] = pa_train_df.apply(loc_to_grid_num, axis=1)

grid_num_counts = pa_train_df['grid_num'].value_counts()
single_grid_locs = grid_num_counts.index[grid_num_counts == 1]

single_null_locations = pa_train_df.index[(pa_train_df.isnull().any(axis=1) & 
                                           pa_train_df.grid_num.isin(single_grid_locs))]

In [16]:
# Find the grid numbers which border the current one
def bordering_grid_nums(grid_num, lat_n_gridlines, lon_n_gridlines):
    n_rows = lat_n_gridlines - 1
    n_cols = lon_n_gridlines - 1
    
    if grid_num < 0:
        raise ValueError("grid number should be non-negative")
    if grid_num > n_rows*n_cols - 1:
        raise ValueError("grid number is too large")
        
    col = grid_num % (n_cols)
    row = grid_num // (n_cols)
    
    left_col, right_col = 0, 3
    top_row, bottom_row = 0, 3
    
    if (col % (n_cols)) == 0:
        left_col = 1
    if (col % (n_cols)) == lon_n_gridlines - 2:
        right_col = 2
    if (row % (n_rows)) == 0:
        top_row = 1
    if (row % (n_rows)) == lat_n_gridlines - 2:
        bottom_row = 2
        
    return [grid_num + n1*n_rows + n2 for n1 in [-1, 0, 1][top_row:bottom_row] for n2 in [-1, 0, 1][left_col:right_col] if (n1 != 0 or n2 != 0)]
    

In [17]:
null_idxs = pa_train_df.index[pa_train_df.isnull().any(axis=1)]
for idx in tqdm(null_idxs):
    row = pa_train_df.loc[idx]
    missing_cols = row.index[row.isnull()]
    
    # If grid cell means contain null values take values from the bordering cells
    groupby_mean_df = pa_train_df.groupby('grid_num')[missing_cols].mean()
    if groupby_mean_df.loc[row.grid_num].isnull().any():
        bordering_cells = bordering_grid_nums(row.grid_num, lat_n_gridlines, lon_n_gridlines)
        bordering_cells = [x for x in bordering_cells if x in groupby_mean_df.index]
        groupby_mean_df.loc[row.grid_num].fillna(groupby_mean_df.loc[bordering_cells, :].mean())

    pa_train_df.loc[idx, missing_cols] = groupby_mean_df.loc[row.grid_num]

100%|██████████| 12390/12390 [01:22<00:00, 150.88it/s]


In [18]:
# Now just fill remaining missing values with the mean
missing_cols = pa_train_df.columns[pa_train_df.isnull().any(axis=0)]
for column in missing_cols:
    pa_train_df[column] = pa_train_df[column].fillna(pa_train_df[column].mean())

In [19]:
# Check for -Inf values
print((pa_train_df == float("-Inf")).any(axis=1).any())
inf_cols = pa_train_df.columns[(pa_train_df == float("-Inf")).any(axis=0)]

for col in inf_cols:
    inf_idxs = pa_train_df.index[pa_train_df[col] == float("-Inf")]
    pa_train_df.loc[inf_idxs, col] = 0

True


# Prediction

In [20]:
# Check that the two dataframes are in the same order w.r.t surveyId
(pa_species_df.surveyId == pa_train_df.surveyId).all()

True

In [21]:
n_species_ids = 5016
target_speciesIds = np.random.choice(pa_metadata_train_df.speciesId.unique(), 
                                     n_species_ids, 
                                     replace=False)

target_speciesIds = np.sort(pa_metadata_train_df.speciesId.unique())

X = pa_train_df.drop(['surveyId', 'speciesCount', 'region', 'country', 'grid_num'], axis=1)
y = pa_species_df.drop('surveyId', axis=1)

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1)

In [23]:
X.head()

,lon,lat,year,country_Andorra,country_Austria,country_Belgium,country_Bosnia and Herzegovina,country_Bulgaria,country_Croatia,country_Czech Republic,...,swir2_10,swir2_9,swir2_8,swir2_7,swir2_6,swir2_5,swir2_4,swir2_3,swir2_2,swir2_1
0,3.099038,43.134956,2021,0,0,0,0,0,0,0,...,17.0,13.0,9.0,17.0,16.0,13.0,11.0,21.0,15.0,13.0
1,9.884560,56.912140,2017,0,0,0,0,0,0,0,...,31.0,34.0,20.0,25.0,28.0,26.0,20.0,20.0,30.0,27.0
2,8.256020,55.637050,2019,0,0,0,0,0,0,0,...,8.0,15.0,9.0,10.0,8.0,15.0,9.0,6.0,7.0,11.0
3,-0.402590,43.505630,2018,0,0,0,0,0,0,0,...,15.0,38.0,35.0,47.0,17.0,23.0,40.0,42.0,15.0,35.0
4,-0.517360,45.806430,2017,0,0,0,0,0,0,0,...,33.0,28.0,38.0,54.0,64.0,42.0,44.0,52.0,32.0,52.0


In [24]:
X.to_csv('X_train.csv', index=False)
y.to_csv('y_train.csv', index=False)